<a href="https://colab.research.google.com/github/Zoe-CHO/Korean-Speech-Emotion-Recognition/blob/main/week4_%EC%A1%B0%EC%B0%AC%EC%96%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
data_fmnist=keras.datasets.fashion_mnist
data_mnist=keras.datasets.mnist

# Getting the In-distribution Data (MNIST)
(X_train, y_train), (X_test, y_test)=data_mnist.load_data()

# Creating Validation and Test set
X_val=X_train[:5000]
y_val=y_train[:5000]

X_train=X_train[5000:]
y_train=y_train[5000:]

# Normalizing the In-distribution Data
X_train=X_train/255
X_test=X_test/255


# Getting the Out-of-distribution Data (Fashion MNIST)
_, (X_ood, y_ood)=data_fmnist.load_data()

# Normalizing the Out-of-distribution Data
X_ood=X_ood/255

# Reshaping the Data
X_train=X_train.reshape(-1, 28, 28, 1)
X_val=X_val.reshape(-1, 28, 28, 1)
X_test=X_test.reshape(-1, 28, 28, 1)
X_ood=X_ood.reshape(-1, 28, 28, 1)

In [ ]:
# Creating a Convolutional Neural Network
model = keras.models.Sequential([
        keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                            input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation="softmax")
])

In [ ]:
# Compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# Training the model on Downstream Task
history=model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
1719/1719 [==============================] - 19s 8ms/step - loss: 1.2580 - accuracy: 0.5722 - val_loss: 20.2550 - val_accuracy: 0.9510
Epoch 2/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.3071 - accuracy: 0.9150 - val_loss: 17.0150 - val_accuracy: 0.9704
Epoch 3/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1951 - accuracy: 0.9488 - val_loss: 11.4919 - val_accuracy: 0.9802
Epoch 4/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1490 - accuracy: 0.9621 - val_loss: 11.9799 - val_accuracy: 0.9834
Epoch 5/10
1719/1719 [==============================] - 11s 7ms/step - loss: 0.1208 - accuracy: 0.9689 - val_loss: 9.3657 - val_accuracy: 0.9862
Epoch 6/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.1019 - accuracy: 0.9736 - val_loss: 12.2360 - val_accuracy: 0.9816
Epoch 7/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.0918 - accuracy: 0.9775 - val_loss: 8.7173 - v

In [ ]:
# Computing Predictions and Maximum Softmax Values of Models
pred_ood = model.predict(X_ood)
pred_test = model.predict(X_test)

maxpred_ood = tf.reduce_max(pred_ood, axis=1)
maxpred_test = tf.reduce_max(pred_test, axis=1)

In [ ]:
# Setting the Threshold for Distinguishing In-distribution/Out-of-distribution Data
threshold = 0.99

# Counting the Number of Data Counted as In-distribution
numid_ood = tf.math.greater(maxpred_ood, threshold)
numid_ood = tf.math.reduce_sum(tf.cast(numid_ood, tf.int32))

numid_test = tf.math.greater(maxpred_test, threshold)
numid_test = tf.math.reduce_sum(tf.cast(numid_test, tf.int32))

# Computing the True Positive Rate and False Positive Rate
num_ood = 10000
num_test = 10000

TPR = numid_test / num_test
FPR = numid_ood / num_ood

print(TPR, FPR)

tf.Tensor(0.9649, shape=(), dtype=float64) tf.Tensor(0.1534, shape=(), dtype=float64)
